In [1]:
# import the libraries
from pandas import read_csv
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot
import numpy
import pandas as pd
import tensorflow as tf
tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [2]:
# load data
df = pd.read_csv('../data/commits_transformed_new.csv', 
                 index_col='date', 
                 parse_dates=['date'],
                 infer_datetime_format=True)

In [3]:
df.shape

(601077, 75)

In [4]:
df = df.drop("total_deletions", axis=1)
df = df.drop("total_additions", axis=1)
df = df.drop("total", axis=1)
df = df.drop("private", axis=1)
df = df.drop("fork", axis=1)
df = df.drop("size", axis=1)
df = df.drop("watchers_count", axis=1)
df = df.drop("has_issues", axis=1)
df = df.drop("has_downloads", axis=1)
df = df.drop("has_wiki", axis=1)
df = df.drop("forks_count", axis=1)
df = df.drop("open_issues_count", axis=1)
df = df.drop("forks", axis=1)
df = df.drop("open_issues", axis=1)
df = df.drop("watchers", axis=1)
df = df.drop("network_count", axis=1)
df = df.drop("type", axis=1)
df = df.drop("admin", axis=1)
df = df.drop("push", axis=1)
df = df.drop("pull", axis=1)
df = df.drop("total_files", axis=1)
df = df.drop("language_C", axis=1)
df = df.drop("language_C#", axis=1)
df = df.drop("language_C++", axis=1)
df = df.drop("language_CSS", axis=1)
df = df.drop("language_CoffeeScript", axis=1)
df = df.drop("language_Go", axis=1)
df = df.drop("language_Java", axis=1)
df = df.drop("language_JavaScript", axis=1)
df = df.drop("language_PHP", axis=1)
df = df.drop("language_Perl", axis=1)
df = df.drop("language_Python", axis=1)
df = df.drop("language_R", axis=1)
df = df.drop("language_Ruby", axis=1)
df = df.drop("language_Scala", axis=1)
df = df.drop("language_Shell", axis=1)
df = df.drop("language_TypeScript", axis=1)

In [5]:
df.isnull().values.any()

False

In [5]:
# split data into X and y
array = df.values
X = array[:,0:37]
y = array[:,37]

In [6]:
# grid search
model = XGBClassifier(
 learning_rate =1,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
n_estimators = [100, 500]
max_depth = [5, 10]
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="roc_auc", n_jobs=-1, cv=kfold, verbose=1)

In [ ]:
# fit the model
grid_result = grid_search.fit(X, y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
# plot results
scores = numpy.array(means).reshape(len(max_depth), len(n_estimators))
for i, value in enumerate(max_depth):
    pyplot.plot(n_estimators, scores[i], label='depth: ' + str(value))
pyplot.legend()
pyplot.xlabel('n_estimators')
pyplot.ylabel('Log Loss')
pyplot.savefig('n_estimators_vs_max_depth.png')